# Phần 1: Thu thập thời tiết các tỉnh thành Việt Nam

### Bước 1: Thiết lập ban đầu

#### 1.0 Chuẩn bị các phụ trợ ban đầu

Nhập các thư viện cần thiết để thực hiện cào dữ liệu

In [1]:
from setupDriver import *
import pandas as pd
import time
from datetime import datetime
import pytz

#### 1.1 Định nghĩa các hàm cào dữ liệu. (ở file setupDriver.py)

__Tùy biến lại dịch vụ gọi trình duyệt Chrome__
- Để tăng hiệu suất thu thập DL, tránh thời gian thừa để tải các loại thành phần không cần thiết của 1 trang web. Tôi đã tùy biến lại driver của selenium. Các driver trong bài này sẽ sử dụng tùy biến này

__Tổng thể, các hàm cào sử dụng một cơ chế như sau:__
- Ra lệnh cho trình duyệt sẽ đợi trong 1 khoảng thời gian cụ thể
- Trong thời gian đợi, trình duyệt sẽ cố gắng tìm,lấy về theo CSS truyền vào:
    + `get_element`: Một phần tử HTML có CSS tương ứng
    + `get_list`: Danh sách phần tử HTML có CSS giống nhau
- Nếu không tìm thấy, trình duyệt sẽ trả về 1 ngoại lệ

__Lý do:__
<p style='text-align: justify;'> Mỗi khi gọi đến một trang web, các phần tử có thể không có mặt ngay lập tức ở file HTML (hay DOM) mà sẽ lần lượt được tải vào DOM dần. Vì vậy, chúng ta phải thiết lập một độ trễ để đợi đến khi phần tử đó có mặt. Hai hàm cào dữ liệu này sẽ đặt ra cơ chế đợi trong 1 khoảng thời gian cho trước, nếu đã tìm thấy thì lập tức dừng lại, không đợi nữa. Vì vậy, nó sẽ tránh được trường hợp ta phải đợi cố định một khoảng thời gian cụ thể mỗi khi tìm phần tử. </p>

__Dưới đây là định nghĩa của hai hàm ấy, nhưng tôi đã viết sẵn bên trong setupDriver.py__

In [48]:
def get_element(driver, selector, timeout=3):
    `return WebDriverWait(driver, timeout).until(presence_of_element_located((By.CSS_SELECTOR, selector)))

In [49]:
def get_list(driver, selector, timeout=3):
    return WebDriverWait(driver, timeout).until(presence_of_all_elements_located((By.CSS_SELECTOR, selector)))

#### 1.2 Lấy ra tên các tỉnh thành Việt Nam ở AccuWeather

AccuWeather có đặc điểm là họ liệt kê đầy đủ 63 tỉnh thành của Việt Nam. Vì vậy, ta sẽ sử dụng selenium:
- Duyệt qua danh sách hiển thị của họ,
- Lấy về tên tỉnh thành và URL đính kèm
- Đưa toàn bộ vào một từ điển tên là `provinces`

In [ ]:
url = "https://www.accuweather.com/vi/browse-locations/asi/vn"
driver = startCrawl()
try:
    driver.get(url)
    results = get_list(driver, ".result-container .search-result")
    provinces = dict()
    for row in results:
        name = row.text
        href = row.get_attribute('href')
        print(name, href)
        provinces[name]=href
except Exception as e:
    print(e)
finally:
    driver.quit()

#### 1.3 Lưu lại danh sách tỉnh thành kèm đường dẫn vào CSV

In [57]:
df = pd.DataFrame(list(provinces.items()), columns=['Tỉnh thành', 'URL'])

In [59]:
len(df)

63

In [60]:
df.to_csv('provinces.csv',encoding='utf-8-sig',index=False)

#### 1.4 Lấy ra danh sách các khu vực thuộc từng tỉnh thành

Vì AccuWeather thậm chí cung cấp thông tin thời tiết của nhiều khu vực trong 1 tỉnh thành. Nên với mỗi một tỉnh thành, ta cần lấy ra danh sách khu vực. Trong đây ta sử dụng selenium làm việc sau:
- Duyệt qua từng phần tử trong `provinces`. Nhớ rằng mỗi phần tử có key là tên tỉnh thành và value là URL
- Mở URL của từng tỉnh thành, quét sạch toàn bộ danh sách khu vực của tỉnh thành đó
- Lấy ra tên và URL của từng khu vực, lưu lại vào 1 danh sách tên `raw_data`

In [61]:
driver = startCrawl()
raw_data = []
for key, value in provinces.items():
    url = value
    try:
        driver.get(url)
        results = get_list(driver, ".result-container .search-result")
        for row in results:
            city_name = row.text
            city_link = row.get_attribute('href')
            record = (key, city_name, city_link)
            raw_data.append(record)
    except Exception as e:
        print(e)
driver.quit()    

#### 1.5 Xử lí danh sách các khu vực

Tuy vậy, mục tiêu bài toán của chúng ta là lấy thông tin thời tiết tổng thể của cả một tỉnh thành, chứ không phải từng khu vực. Nên chúng ta phải xử lí một chút dữ liệu trên. Đầu tiên, ta lưu lại danh sách `raw_data` vào một DataFrame tên `df2`, lưu tiếp vào 1 file csv.

In [62]:
df2 = pd.DataFrame(raw_data, columns=['Tỉnh thành','Khu vực','URL'])

In [63]:
df2.to_csv('area.csv',encoding='utf-8-sig',index=False)

Một số tỉnh thành đã tồn tại sẵn __trang tổng quan__ của chính tỉnh thành đó.
- VD: Danh sách khu vực của tỉnh thành `Hà Nội` đã có sẵn một khu vực cũng tên `Hà Nội`.
- Link danh sách: https://www.accuweather.com/vi/browse-locations/asi/vn/hn

Một số tỉnh thành không có __trang tổng quan__.
- VD: Tỉnh thành `An Giang` không có trang nào cũng tên `An Giang`
- Link danh sách: https://www.accuweather.com/vi/browse-locations/asi/vn/44

__Hướng giải quyết__: 
 - Loại bỏ các trang khu vực khác, chỉ giữ lại __trang tổng quan__ với các tỉnh thành có __trang tổng quan__
 - Giữ lại toàn bộ các trang khu vực của các tỉnh thành mà không có __trang tổng quan__

In [98]:
df2 = pd.read_csv('area.csv',encoding='utf-8-sig')
df2.head()

In [122]:
general = df2[df2['Tỉnh thành'] == df2['Khu vực']]['Tỉnh thành'].values

def check_condition(x):
    same_general = x['Tỉnh thành'] == x['Khu vực']
    not_general = x['Tỉnh thành'] not in general
    return same_general or not_general

df3 = df2[df2.apply(check_condition, axis=1)]

Trường hợp đặc biệt:
- Một số tỉnh thành có tồn tại __trang tổng quan__, nhưng tên lại không trùng với tên tỉnh thành do lỗi chính tả, sai trên CSDL AccuWeather
- VD: Hồ Chí Minh có trang tổng quan là Thành phố Hồ Chí Minh

Hướng giải quyết:
- Loại bỏ các trang khu vực thừa, chỉ giữ lại trang tổng quan với Hồ Chí Minh

In [123]:
df3 = df3[~((df3['Tỉnh thành'] == 'Hồ Chí Minh') & (df3['Khu vực'] != 'Thành phố Hồ Chí Minh'))]
df3.reset_index(inplace=True, drop=True)

In [127]:
len(df3)

298

In [128]:
df3.to_csv('area.csv',encoding='utf-8-sig',index=False)

### Bước 2: Thu thập dữ liệu của từng tỉnh thành (liên tục trong 1 tuần)

Sau khi thu thập, dữ liệu sẽ được lưu lại vào 1 file csv

In [17]:
# Trong quá trình đợi cào dữ liệu, bạn có thể nghe nhạc nền này để giết thời gian :)
import IPython
IPython.display.Audio("bgm.mp3")

In [1]:
from setupDriver import *
import pandas as pd
import time
from datetime import datetime
import pytz

In [15]:
columns = ['ThoiGian','TinhThanh','KhuVuc','NhietDo (°C)','LuongMua (mm)',
           'DoAm (%)','DoAmTrongNha (%)','KhiAp (mb)',
           'MatDoMay (%)','TamNhin (km)','AQI','TrangThaiKhongKhi',
           'PM2_TrangThai','PM2_ChiSo','PM2_Matdo(µg/m³)',
           'PM10_TrangThai','PM10_ChiSo','PM10_Matdo(µg/m³)',
           'SO2_TrangThai','SO2_ChiSo','SO2_Matdo(µg/m³)',
           'NO2_TrangThai','NO2_ChiSo','NO2_Matdo(µg/m³)']
df3 = pd.read_csv('area.csv',encoding='utf-8-sig')
vietnam_tz = pytz.timezone('Asia/Ho_Chi_Minh')
temp_data = df3.iterrows()
start = datetime.now()
raw_thing = []
labels_of_interest = {'Humidity': None, 'Indoor Humidity': None, 'Pressure': None, 'Visibility': None}

#Bắt đầu cào dữ liệu từ đây thông qua 1 vòng lặp, đi qua toàn bộ khu vực
try:
    driver = startCrawl()
    for index, row in temp_data:
        #Tạo bản ghi tạm
        record = []

        #Đưa vào bản ghi thời gian hiện tại, tỉnh thành, khu vực
        record.append(datetime.now(vietnam_tz))
        record.append(row['Tỉnh thành'])
        record.append(row['Khu vực'])

        #Duyệt đường dẫn của khu vực đó và chuẩn bị link chất lượng không khí
        driver.get(row['URL'])
        a = get_element(driver, ".air-quality-module-wrapper")
        link = a.get_attribute("href")

        #Lấy các thông số chi tiết về thời tiết, nếu gặp lỗi không load đc link, refresh lại link đó
        try:
            get_element(driver, ".cur-con-weather-card").click()
        except Exception as e:
            driver.refresh()
        temp = get_element(driver, ".display-temp").text
        record.append(temp)

        #Lấy thông số về lượng mưa, tính trung bình lượng mưa Day và Night
        panel_items = driver.find_elements(By.CSS_SELECTOR, "p.panel-item")
        count = 0
        rain = 0
        for item in panel_items:
            if "mm" in item.text and count % 2 == 0:
                span_value = item.find_element(By.CSS_SELECTOR, "span.value").text.split()
                rain += float(span_value[0])
                count += 1
        if count > 0:
            rain = rain/count
        record.append(rain)

        # Lấy thông tin về các chỉ số thời tiết còn lại
        details = get_list(driver, ".current-weather-details .detail-item.spaced-content")
        for item in details[2:]:
            label = item.find_element(By.CSS_SELECTOR,"div:first-child").text
            value = item.find_element(By.CSS_SELECTOR, "div:last-child").text
            if label in ('Humidity','Indoor Humidity','Pressure','Cloud Cover','Visibility'):
                record.append(value)

        # Lấy ra thông tin về chất lượng không khí từ link đã chuẩn bị
        driver.get(link)
        aqi = get_element(driver, ".aq-number").text
        record.append(aqi)
        category = get_element(driver,".category-text").text
        record.append(category)

        details_pol = get_list(driver, "#pollutants .air-quality-pollutant")
        for item in details_pol[:-2]:
            cdt = item.find_element(By.CSS_SELECTOR, ".display-type sub")
            conditional = cdt.get_attribute("conditional") + cdt.text
            category = item.find_element(By.CSS_SELECTOR, ".category").text
            column_left = item.find_element(By.CSS_SELECTOR, ".desktop-left")
            pollutant_index = column_left.find_element(By.CSS_SELECTOR, ".pollutant-index").text
            pollutant_concentration = column_left.find_element(By.CSS_SELECTOR, ".pollutant-concentration").text
            record.append(category)
            record.append(pollutant_index)
            record.append(pollutant_concentration)
        raw_thing.append(record)
        print(f"{row['Khu vực']} - {row['Tỉnh thành']}",end=', ')
except Exception as e:
    print(f"Cào thất bại!, lỗi: {e}")
else:
    driver.quit()
    end = datetime.now()
    td = (end - start).total_seconds() / 60
    print(f"Cào thành công, thời gian: {td:.03f} phút")
    df_new = pd.DataFrame(raw_thing, columns=columns)
    today = datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')).date()
    df_new.to_csv(f'./raw_data/{today}.csv',encoding='utf-8-sig',index=False)

Châu Đốc - An Giang, Chợ Mới - An Giang, Long Xuyên - An Giang, Phú Châu - An Giang, Tịnh Biên - An Giang, Tri Tôn - An Giang, Xóm Khánh Hòa - An Giang, Ấp Bình Châu - Bà Rịa - Vũng Tàu, Bà Rịa - Bà Rịa - Vũng Tàu, Châu Thành - Bà Rịa - Vũng Tàu, Chợ Phước Hải - Bà Rịa - Vũng Tàu, Cỏ Ống - Bà Rịa - Vũng Tàu, Đất Đỏ - Bà Rịa - Vũng Tàu, Grande Village de Poulo Condore - Bà Rịa - Vũng Tàu, Long Lễ - Bà Rịa - Vũng Tàu, Vũng Tàu - Bà Rịa - Vũng Tàu, Xã Bình Giã - Bà Rịa - Vũng Tàu, Bắc Giang - Bắc Giang, Bắc Kạn - Bắc Kạn, Bạc Liêu - Bạc Liêu, Bắc Ninh - Bắc Ninh, Bến Tre - Bến Tre, An Lão - Bình Định, Bồng Sơn - Bình Định, Chấn An - Bình Định, Định Bình - Bình Định, Dương Liễu - Bình Định, Hữu Giang - Bình Định, Phú An - Bình Định, Phù Cát - Bình Định, Phú Xuân - Bình Định, Quy Nhơn - Bình Định, Tam Quan - Bình Định, Vân Canh - Bình Định, Văn Thiên - Bình Định, Vĩnh Phúc - Bình Định, Ấp Bàu Bàng - Bình Dương, Dầu Tiếng - Bình Dương, Dĩ An - Bình Dương, Tân Uyên - Bình Dương, Thủ Dầu Một -

### Bước 3: Giải thích tập DL thu thập được

In [16]:
import pandas as pd
df = pd.read_csv(f'./raw_data/{today}.csv',encoding='utf-8-sig')
pd.set_option('display.max_columns', None)
df.head(2)

,ThoiGian,TinhThanh,KhuVuc,NhietDo (°C),LuongMua (mm),DoAm (%),DoAmTrongNha (%),KhiAp (mb),MatDoMay (%),TamNhin (km),AQI,TrangThaiKhongKhi,PM2_TrangThai,PM2_ChiSo,PM2_Matdo(µg/m³),PM10_TrangThai,PM10_ChiSo,PM10_Matdo(µg/m³),SO2_TrangThai,SO2_ChiSo,SO2_Matdo(µg/m³),NO2_TrangThai,NO2_ChiSo,NO2_Matdo(µg/m³)
0,2024-06-16 08:37:21.617904+07:00,An Giang,Châu Đốc,29°C,12.0,77%,77% (Extremely Humid),↔ 1006 mb,90%,10 km,27,Fair,Fair,27,7 µg/m³,Excellent,16,12 µg/m³,Excellent,10,30 µg/m³,Excellent,6,3 µg/m³
1,2024-06-16 08:37:27.446566+07:00,An Giang,Chợ Mới,28°C,13.4,84%,84% (Extremely Humid),↔ 1006 mb,50%,8 km,27,Fair,Fair,27,7 µg/m³,Excellent,16,12 µg/m³,Excellent,11,32 µg/m³,Excellent,5,3 µg/m³


#### 3.1 Tổng quan tập dữ liệu
- Mỗi tập dữ liệu ghi lại các thông tin chỉ số về thời tiết __THỰC SỰ__ và chất lượng không khí trong 1 ngày của toàn bộ các tỉnh thành.
- Chúng tôi thu thập các tập DL vào 1 mốc thời gian cố định mỗi ngày (bắt đầu từ 7/6/2024). Dự kiến sẽ thu thập 7 tập dữ liệu tương ứng với 7 ngày.
- Một số tỉnh thành không có bản ghi tổng quan (do AccuWeather không cung cấp trang tổng quan - xem lại nội dung Chương I), nên ta có thể thấy 1 tỉnh thành có nhiều bản ghi khu vực khác nhau.

#### 0.2 Thông tin từng cột trong tập DL
__Thông tin khu vực__
- `ThoiGian`: thời gian thu thập dữ liệu từ AccuWeather của bản ghi
- `TinhThanh`: tên tỉnh thành của bản ghi đó
- `KhuVuc`: tên khu vực của bản ghi. Trong chương 1 ta đã biết:
    + Nếu tên khu vực trùng tên tỉnh thành, tức là bản ghi này là bản ghi tổng quan của 1 tỉnh thành 
    + Ngược lại, tỉnh thành này không có trang tổng quan nên ta mới phải thu thập toàn bộ các khu vực khác của nó
    
__Thông tin chính về thời tiết__

- `NhietDo (°C)`: nhiệt độ thực tế được AccuWeather cập nhật thời gian thực
- `LuongMua (mm)`: trung bình lượng mưa ngày và đêm dự kiến của ngày đó
- `DoAm (%)`: độ ẩm đo được trong thời gian thực
- `DoAmTrongNha (%)`: mức độ khô và độ ẩm bên trong một ngôi nhà hoặc tòa nhà 
    + Đây là một phần của công nghệ đo đạc mới của AccuWeather. 
    + Độ ẩm trong nhà có mối liên hệ với khả năng lây lan của 1 số loại bệnh (VD: COVID-19)
    + Tìm hiểu thêm: [xem tại đây](https://www.accuweather.com/en/press/accuweather-com-introduces-accuweather-indoor-humidity-for-better-health-safety-and-comfort/744074)
    
__Thông tin khác về thời tiết__
- `KhiAp (mb)`: đo lường áp suất của không khí trong môi trường
    + Ý nghĩa : tham gia báo hiệu, tìm hiểu sự hình thành áp cao, áp thấp nhiệt đới, bão
- `MatDoMay (%)`: đo độ bao phủ của mây trên bầu trời. Nó có các ý nghĩa sau:
    + Nghiên cứu khí hậu: xem xét cách mây ảnh hưởng tới hiệu ứng nhà kính, phản ánh sự biến đổi khí hậu
    + Trong hàng không: độ bao phủ của nó ảnh hưởng tới cách xây dựng đường bay an toàn, quyết định hạ cánh,..
    + Ngoài ra mật độ mây cũng hữu ích với các lĩnh vực như Nông nghiệp, Năng lượng tái tạo..
- `TamNhin (km)`: tầm nhìn xa nhất có thể quan sát được.

__Thông tin chính về chất lượng không khí__
- `AQI`: Chỉ số chất lượng không khí thực tế trong ngày đó. Được tính bằng công thức dựa theo chỉ số của 1 số loại khí
    + Lưu ý: AQI được tính bằng nhiều chỉ số không khí khác ngoài các chỉ số được thống kê trong tập DL này
- `TrangThaiKhongKhi`: nhận xét/đánh giá của AccuWeather đưa ra về AQI hiện tại.

__Thông tin chi tiết về các kiểu khí có hại trong bầu không khí__
- `Tên khí_TrangThai`: nhận xét/đánh giá của AccuWeather đưa ra về giá trị đo đạc của thành phần này
- `Tên khí_Chiso`: chỉ số/độ phủ đo được của loại khí đó. Chỉ số càng lớn tức trong không khí càng có nhiều khí hại này.
- `Tên khí_MatDo (µg/m³)`: mật độ phủ của loại khí này bên trong không khí mỗi 1 đơn vị.